In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import shap
import numpy as np
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
# Configuration
access_token = ""  # Replace with your token
model_name = "meta-llama/Llama-3.1-8B-Instruct"
max_length = 256  # Reduced sequence length for stability

# Label mapping
label_map = {
    0: 'entailment',
    1: 'neutral',
    2: 'contradiction'
}

# Load model and tokenizer
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token,
                                           torch_dtype=torch.float16,
                                           device_map="auto")

# Fix tokenizer settings
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model.eval()

Loading model...


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [ ]:
# Load dataset
num_samples = 500
print("Loading dataset...")
dataset = load_dataset("esnli", split=f"validation[:{num_samples}]")

Loading dataset...


In [ ]:
# Improved prediction function
def safe_predict(texts):
    try:
        if isinstance(texts, str):
            texts = [texts]
        elif isinstance(texts, np.ndarray):
            texts = texts.tolist()

        # Ensure we have a list of strings
        if not all(isinstance(t, str) for t in texts):
            texts = [str(t) for t in texts]

        inputs = tokenizer(texts, return_tensors="pt",
                         padding=True, truncation=True,
                         max_length=max_length).to(model.device)

        with torch.no_grad():
            outputs = model(**inputs)

        # Return logits for the last token only
        return outputs.logits[:, -1, :3].cpu().numpy()  # Only first 3 dimensions for our labels

    except Exception as e:
        print(f"\nPrediction error: {str(e)}")
        return np.zeros((len(texts), 3))  # Return neutral predictions on error


In [ ]:
# Robust SHAP explainer
def get_shap_values(text):
    try:
        # Create explainer with safety checks
        masker = shap.maskers.Text(tokenizer, mask_token=tokenizer.eos_token)
        explainer = shap.Explainer(
            safe_predict,
            masker,
            algorithm="permutation",
            max_evals=200,  # Reduced for speed
            output_names=list(label_map.values()))

        with torch.no_grad():
            shap_values = explainer([text])
        return shap_values
    except Exception as e:
        print(f"\nSHAP error: {str(e)}")
        return None


In [ ]:
# Robust explanation generator
def generate_explanation(prompt, max_new_tokens=100):
    try:
        inputs = tokenizer(prompt, return_tensors="pt",
                         max_length=max_length,
                         truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )

        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the new generated part
        input_length = inputs.input_ids.shape[1]
        return full_text[input_length:].strip()
    except Exception as e:
        print(f"\nGeneration error: {str(e)}")
        return "Error generating explanation"

In [ ]:
# Main processing loop
results_c = []
results_ic = []
print(f"\nProcessing {num_samples} samples...")
start_time = time.time()

for i, example in enumerate(tqdm(dataset, desc="Processing samples")):
    if i >= num_samples:
        break

    premise = example['premise']
    hypothesis = example['hypothesis']
    text = f"Premise: {premise}\nHypothesis: {hypothesis}"
    gt_label = label_map[example['label']]

    # Get prediction
    pred_prompt = f"""Classify this as entailment, neutral, or contradiction:

Premise: {premise}
Hypothesis: {hypothesis}
Answer:"""

    pred_label = generate_explanation(pred_prompt, max_new_tokens=10)
    pred_label = pred_label.lower().strip()

    # Clean and validate prediction
    pred_label = pred_label.replace('"', '').replace("'", "")
    if "entail" in pred_label:
        pred_label = "entailment"
    elif "neutral" in pred_label:
        pred_label = "neutral"
    elif "contradict" in pred_label:
        pred_label = "contradiction"
    else:
        pred_label = "neutral"  # Default fallback

    print(f"{gt_label} --- {pred_label}")

    # Get SHAP values
    shap_values = get_shap_values(text)
    shap_str = str([(token, values.tolist()) for token, values in zip(shap_values.data[0], shap_values.values[0])])

    if pred_label == gt_label:
      # Store results
      results_c.append({
        'premise': premise,
        'hypothesis': hypothesis,
        'gt_label': gt_label,
        'pred_label': pred_label,
        'shap_value': shap_str,
        'human_explanation': example.get('explanation_1', '')
      })

    else:
      # Store results
      results_ic.append({
        'premise': premise,
        'hypothesis': hypothesis,
        'gt_label': gt_label,
        'pred_label': pred_label,
        'shap_value': shap_str,
        'human_explanation': example.get('explanation_1', '')
      })



Processing 500 samples...


Processing samples:   0%|          | 0/500 [00:00<?, ?it/s]

neutral --- entailment


Processing samples:   0%|          | 1/500 [00:00<08:11,  1.01it/s]

entailment --- contradiction


...

Processing samples: 100%|█████████▉| 498/500 [10:33<00:02,  1.23s/it]

entailment --- entailment


Processing samples: 100%|█████████▉| 499/500 [10:34<00:01,  1.23s/it]

entailment --- entailment


Processing samples: 100%|██████████| 500/500 [10:35<00:00,  1.27s/it]


In [ ]:
df_c = pd.DataFrame(results_c)
df_ic = pd.DataFrame(results_ic)

In [ ]:
# Save results
output_file_c  = "llama3_nli_analysis_500_shap_correct.csv"
output_file_ic = "llama3_nli_analysis_500_shap_incorrect.csv"

df_c.to_csv(output_file_c, index=False)
df_ic.to_csv(output_file_ic, index=False)

In [ ]:
df_c.shape

(179, 6)